In [1]:
import sagemaker
import boto3 
from sagemaker.image_uris import retrieve
from sagemaker.pytorch import PyTorch, PyTorchModel
import os 
from PIL import Image
import json
from datetime import datetime
from sagemaker.multidatamodel import MultiDataModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/dev/.config/sagemaker/config.yaml


In [ ]:
sm_boto3 = boto3.client("sagemaker")

sess = sagemaker.Session()

region = sess.boto_session.region_name

In [3]:
bucket = os.getenv("BUCKET")
role_arn = os.getenv("ROLE_ARN")

In [4]:

if not bucket  and role_arn:
    
    raise ValueError("Enviroment Variables are not configured")

else: print("Variables Loaded sucesfully")

Variables Loaded sucesfully


In [11]:
# training_data =  f'{bucket}data'"
training_data = f'{bucket}mxnet_data'
s3_output_path = f'{bucket}output'

# Train

### VGG16 & Resnet18

In [ ]:
hyperparameters={
    'epochs': 10,                       
    'batch_size': 48,                   
    'learning_rate': 0.0003,            
    'weight_decay': 0.00005,            # Regularización ligera
    'early_stopping': True,
    'use_sampler': True,                # Crítico para detectar defectos
    'augmentation_intensity': 0.8,      # Máximo aumento para casos raros
    'dropout_rate': 0.4,                # Regularización moderada
    'patience': 10,                     # Más tolerancia para aprendizaje fino
    # 'optimizer': 'AdamW',             # Optimizador recomendado
    # 'scheduler': 'ReduceLROnPlateau', # Ajuste automático de LR
    'val_split': 0.2,                  
    'threshold': 0.85                   # Umbral de confianza para producción
    }

In [ ]:
# model_name = 'VGG16'
model_name = 'Resnet18'
model_name = 'SimpleCNN'


In [ ]:
estimator = PyTorch(
    entry_point='train.py',  
    source_dir='src',
    role=role_arn,
    instance_count=1,  
    instance_type='ml.g4dn.xlarge', 
    # instance_type ='ml.m5.large',
    framework_version='1.12.0', 
    py_version='py38',  
    hyperparameters=hyperparameters,
    base_job_name=f"{model_name}-{datetime.now().strftime('%Y%m%d-%H%M')}",
    output_path= bucket + "output",  
    sagemaker_session=sess,
    use_spot_intances=True,
    # max_wait=7200,
    max_run=3600,
)

In [ ]:

estimator.fit(training_data)


### AWS MXnet 

In [ ]:
hyperparams = {
    "num_classes": 2,
    "num_training_samples": 4000,  
    "image_shape": "3,224,224",    
    "epochs": 20,
    "mini_batch_size": 48,
    "learning_rate": 0.0003,
    "use_pretrained_model": 1,
    'weight_decay': 0.00005,           
    'early_stopping': True

}

In [ ]:
from sagemaker.mxnet import MXNet

estimator = MXNet(
    entry_point='train.py',
    framework_version='1.9.0',
    py_version='py38',
    hyperparameters=hyperparams
)

In [13]:
model_name = "MXnet"

training_image = retrieve(framework="image-classification", region=region, version="latest")

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [ ]:

estimator = sagemaker.estimator.Estimator(
    training_image,
    role_arn,
    instance_count=1,
    instance_type='ml.g4dn.xlarge', 
    # instance_type = 'ml.p3.2xlarge,'
    # instance_type ='ml.m5.large',
    base_job_name=f"{model_name}-{datetime.now().strftime('%Y%m%d-%H%M')}",
    hyperparameters=hyperparams,
    output_path=s3_output_path
)
    

In [17]:
s3_train_imgs = f"{training_data}/train"
s3_valid_imgs = f'{training_data}/validation'
s3_train_annot = f'{training_data}/train_lst'
s3_valid_annot = f'{training_data}/validation_lst'

train_imgs = sagemaker.inputs.TrainingInput(
    s3_train_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
valid_imgs = sagemaker.inputs.TrainingInput(
    s3_valid_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
train_annot = sagemaker.inputs.TrainingInput(
    s3_train_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
valid_annot = sagemaker.inputs.TrainingInput(
    s3_valid_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)

data_channels = {
    "train": train_imgs,
    "validation": valid_imgs,
    "train_lst": train_annot,
    "validation_lst": valid_annot,
}

In [18]:
estimator.fit(data_channels, logs=True)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: MXnet-20250524-0037-2025-05-24-05-47-56-804


2025-05-24 05:48:02 Starting - Starting the training job...
2025-05-24 05:48:18 Starting - Preparing the instances for training...
2025-05-24 05:48:45 Downloading - Downloading input data...
2025-05-24 05:49:20 Downloading - Downloading the training image...............
2025-05-24 05:51:53 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
Nvidia gpu devices, drivers and cuda toolkit versions (only available on hosts with GPU):
Sat May 24 05:52:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |          

Training seconds: 1721
Billable seconds: 1721


# Deploy


## Multimodel

In [ ]:
model_uris = {
    "VGG16": f'{s3_output_path}/VGG16-20250519-1337-2025-05-19-18-37-49-859/output/model.tar.gz',
    "ResNet": f'{s3_output_path}/pytorch-training-2025-05-19-18-03-50-824/output/model.tar.gz',
    "SimpleCNN":   f'{s3_output_path}/SimpleCNN-20250519-1700-2025-05-19-22-00-59-316/output/model.tar.gz'
}
ensemble_models = {'MODEL_PATHS':
                f'{s3_output_path}/VGG16-20250519-1337-2025-05-19-18-37-49-859/output/model.tar.gz,'
                f'{s3_output_path}/pytorch-training-2025-05-19-18-03-50-824/output/model.tar.gz,'
                f'{s3_output_path}/SimpleCNN-20250519-1700-2025-05-19-22-00-59-316/output/model.tar.gz',
                'MODEL_WEIGHTS': '0.2,0.5,0.3'}


In [ ]:
boto_session = boto3.Session(region_name=region)
sagemaker_session = sagemaker.Session(boto_session=boto_session)

sagemaker_runtime = boto3.client(
    "sagemaker-runtime", region_name=region)


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [62]:
image_uri =	"763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:2.0.0-gpu-py310-cu118-ubuntu20.04-sagemaker"

In [ ]:
model = PyTorchModel(
    model_data='s3://pitahaya-classification/placehordes/placeholder.tar.gz',
    role=role_arn,
    entry_point="inference.py",
    framework_version='2.0.0',
    py_version='py39',
    source_dir='code',
    image_uri = image_uri,
    env=ensemble_models,
    sagemaker_session=sagemaker_session
)

In [ ]:
mme = MultiDataModel(
    name="PitahayaEnsemble",
    model_data_prefix=f"{bucket}ensemble_models/",
    model=model,
    sagemaker_session=sagemaker_session
)

In [ ]:


for model_name, model_uri in model_uris.items():
    mme.add_model(
        model_data_source=model_uri,  
        model_data_path=model_name    
    )

In [1]:
endpoint_name="Pitahaya-classification-endpoint"

In [ ]:

predictor = mme.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.xlarge',
    endpoint_name=endpoint_name
)

-------------!

In [ ]:
# Agregar nuevo modelo
mme.add_model(model_data_source=f"{bucket}/output/model/modelo.tar.gz", model_data_path="nuevo-modelo")

In [66]:
predictor.delete_endpoint()

# Inference

In [ ]:
image_path = 'data/normal/Fresh_Dragon_Augmented_Data0003.jpg'
image = Image.open(image_path)
image = image.resize((224, 224))  


In [ ]:
# Hacer la predicción llamando al endpoint
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/jpeg" 
)

In [ ]:
result = json.loads(response['Body'].read().decode())
print(f"Resultado de la predicción: {result}")

In [ ]:
# Función para visualizar imágenes con predicciones
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)


In [ ]:
# Visualización de algunas imágenes con predicciones
model.eval()
inputs, classes = next(iter(dataloaders['val']))
inputs = inputs.to(device)
classes = classes.to(device)

outputs = model(inputs)
_, preds = torch.max(outputs, 1)

plt.figure(figsize=(15, 15))
for i in range(min(batch_size, len(inputs))):
    ax = plt.subplot(min(batch_size, len(inputs)) // 8, 8, i + 1)
    ax.axis('off')
    ax.set_title(f'predicted: {class_names[preds[i]]}\ntrue: {class_names[classes[i]]}')
    imshow(inputs.cpu().data[i])

plt.show()


# AUTOSCALING


In [ ]:

autoscaling = boto3.client('application-autoscaling')

resource_id = f"endpoint/pitahaya-ensemble-endpoint/variant/AllTraffic"

autoscaling.register_scalable_target(
    ServiceNamespace='sagemaker',
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    MinCapacity=1,
    MaxCapacity=4
)